<a href="https://colab.research.google.com/github/mitrasurabhi/NLP-PROJECTS/blob/main/word_to_vec_CNN_quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
nltk.download("popular")
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from wordcloud import WordCloud
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense,Bidirectional,GlobalMaxPool1D, Input, GRU
from tensorflow.keras.models import Model,Sequential
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import gensim

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
dataset = pd.read_csv('https://cdn.extras.talentsprint.com/ADSMI/Datasets/quora_question_pairs.zip')
dataset.drop(['id','qid1','qid2'],axis=1,inplace=True)
dataset.dropna(inplace= True)
dataset.tail()

,question1,question2,is_duplicate
404285,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,What is one coin?,What's this coin?,0
404288,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0
404289,What is like to have sex with cousin?,What is it like to have sex with your cousin?,0


In [ ]:
CONTRACTION_MAP = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

In [ ]:
def cleaning_text(text):
    text = str(text).lower()
#text = re.sub(r'\b(\w+)[^\w\s]*\b', r'\1', text)
 #remove punctuations
    reg = re.compile('([^0-9A-Za-z t])|(\w+://\S+)|([.\/@#*<>?])')
    text=re.sub(reg,' ',text)
    # expanding the words in using contraction map
    new_text=''
    for word in text.split():
        if word in CONTRACTION_MAP.keys():
            new_text=new_text+CONTRACTION_MAP[word]
        else:
            new_text=new_text+word
        new_text=new_text+' '
    text=new_text
    #remove stop words
    stop_words=set(stopwords.words('english'))
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])

    return text

In [ ]:
quora = dataset.copy()
quora.question1=dataset.question1.apply(cleaning_text)
quora.question2=dataset.question2.apply(cleaning_text)

In [ ]:
quora.tail()

,question1,question2,is_duplicate
404285,many keywords racket programming language late...,many keywords perl programming language latest...,0
404286,believe life death,true life death,1
404287,one coin,coin,0
404288,approx annual cost living studying uic chicago...,little hairfall problem want use hair styling ...,0
404289,like sex cousin,like sex cousin,0


In [ ]:
# Tokenize the text data
tokenizer = Tokenizer()
clean_ques = list(quora['question1'])+list(quora['question2'])
tokenizer.fit_on_texts(clean_ques)
new_df = pd.DataFrame()
new_df['question1'] = tokenizer.texts_to_sequences(quora["question1"])
new_df['question2'] = tokenizer.texts_to_sequences(quora["question2"])

vocab_size = len(tokenizer.word_index) + 1
print('vocab_size : ' , vocab_size)

vocab_size :  77666


In [ ]:
new_df

,question1,question2
0,"[563, 563, 2063, 487, 495, 262, 3]","[563, 563, 2063, 487, 495, 262]"
1,"[269, 12353, 11737, 18267, 3125]","[8, 119, 21, 156, 9845, 12353, 11737, 18267, 3..."
2,"[147, 352, 277, 1425, 87, 2512]","[277, 352, 3018, 1228, 9333]"
3,"[2513, 2489, 568]","[35, 3598, 135, 2024, 1222, 135, 2229, 1222, 2..."
4,"[9, 5929, 162, 30097, 1718, 1767, 9192, 1758, ...","[1738, 8, 1135, 1767, 162]"
...,...,...
404282,"[30, 2873, 8640, 109, 65, 1033, 828]","[30, 2873, 10032, 109, 65, 1033, 828]"
404283,"[259, 12, 427]","[192, 12, 427]"
404284,"[9, 1748]",[1748]
404285,"[6309, 3027, 201, 372, 588, 25715, 2949, 21, 68]","[862, 7435, 189, 37, 23, 195, 17794, 196, 9, 9..."


In [ ]:
MAX_SEQ1 = np.max(new_df['question1'].apply(lambda x :len(x)))
MAX_SEQ2 = np.max(new_df['question2'].apply(lambda x :len(x)))

print('max words in sentence1 : ' , MAX_SEQ1)
print('max words in sentence2 : ' , MAX_SEQ2)

max words in sentence1 :  54
max words in sentence2 :  103


In [ ]:
list1 = list(new_df['question1'])
df1 = pd.DataFrame(list1)
df1 = df1.fillna('0')
print(df1.shape)

list2 = list(new_df['question2'])
df2 = pd.DataFrame(list2)
df2 = df2.fillna('0')
print(df2.shape)


(404287, 54)
(404287, 103)


In [ ]:
temp_df = pd.concat([df1, df2], axis=1)
temp_df = temp_df.astype(int)
temp_df


,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,101,102
0,563,563,2063,487,495,262,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,269,12353,11737,18267,3125,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,147,352,277,1425,87,2512,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2513,2489,568,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,5929,162,30097,1718,1767,9192,1758,9494,5470,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404282,30,2873,8640,109,65,1033,828,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404283,259,12,427,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404284,9,1748,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404285,6309,3027,201,372,588,25715,2949,21,68,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print('vocab_size : ' , vocab_size)
temp_df.shape
max_length = temp_df.shape[1]
print('max_length : ' , max_length)

vocab_size :  77666
max_length :  157


In [ ]:
# Train the word2vec model
#from gensim.models import Word2Vec
sentence1 = [str(sentence).split() for sentence in quora['question1']]
sentence2 = [str(sentence).split() for sentence in quora['question2']]
sentence = sentence1 + sentence2
#sentence1[:20]
w2v_model = gensim.models.Word2Vec(sentence,vector_size=100,window=5,min_count=1)

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

embedding_matrix[:5]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+0

In [ ]:
#Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df,quora['is_duplicate'],test_size=0.2, random_state=50)

#print(f'X train is: \n{X_train}')
print(f'X train shape: {X_train.shape}\n')

#print(f'X test is: \n{X_test}')
print(f'X test shape: {X_test.shape}\n')

#print(f'y train is: \n{y_train}')
print(f'y train shape: {y_train.shape}\n')

#print(f' y test is: \n{y_test}')
print(f'y test shape: {y_test.shape}\n')

X train shape: (323429, 157)

X test shape: (80858, 157)

y train shape: (323429,)

y test shape: (80858,)



In [ ]:
# Define the CNN model

model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
10108/10108 [==============================] - 616s 61ms/step - loss: 0.5061 - accuracy: 0.7449 - val_loss: 0.4787 - val_accuracy: 0.7606
Epoch 2/10
10108/10108 [==============================] - 546s 54ms/step - loss: 0.4569 - accuracy: 0.7758 - val_loss: 0.4599 - val_accuracy: 0.7736
Epoch 3/10
10108/10108 [==============================] - 538s 53ms/step - loss: 0.4299 - accuracy: 0.7906 - val_loss: 0.4617 - val_accuracy: 0.7721
Epoch 4/10
10108/10108 [==============================] - 539s 53ms/step - loss: 0.4073 - accuracy: 0.8039 - val_loss: 0.4593 - val_accuracy: 0.7766
Epoch 5/10
10108/10108 [==============================] - 539s 53ms/step - loss: 0.3889 - accuracy: 0.8137 - val_loss: 0.4620 - val_accuracy: 0.7830
Epoch 6/10
10108/10108 [==============================] - 539s 53ms/step - loss: 0.3721 - accuracy: 0.8234 - val_loss: 0.4539 - val_accuracy: 0.7816
Epoch 7/10
 8069/10108 [======================>.......] - ETA: 1:42 - loss: 0.3545 - accuracy: 0.8332

In [ ]:
loss, accuracy= model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

In [ ]:
cnn_pred = np.round(model.predict(X_test))

In [ ]:
# Precision , Recall , F1-score
cr_lstm = classification_report(y_test,cnn_pred)
print(cr_lstm)

In [ ]:
# # RNN/GRU model
# model_GRU = Sequential([
#     Embedding(vocab_size, 100, \
#               embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),\
#               input_length=max_length, trainable=False),
#     Bidirectional(tf.keras.layers.GRU(256, return_sequences=True)),
#     tf.keras.layers.GlobalAveragePooling1D(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(1, activation='softmax')
# ])
# model_GRU.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# print(model.summary())

In [ ]:
# tf.keras.utils.plot_model(model_GRU)

In [ ]:
# history_GRU = model_GRU.fit(X_train, y_train, batch_size=100, epochs=5, validation_split=0.1)

In [ ]:
# loss_GRU, accuracy_GRU= model_GRU.evaluate(X_test, y_test)
# print(f"Test loss: {loss}")
# print(f"Test accuracy: {accuracy}")

In [ ]:
# lstm_pred = np.round(model_GRU.predict(X_test))

In [ ]:
# # Precision , Recall , F1-score
# cr_lstm = classification_report(y_test,lstm_pred)
# print(cr_lstm)